In [1]:
"""
@author: brianwsyang
@date: Sat Jul 27, 2019
"""

import pandas as pd
import numpy as np
import re

In [46]:
num_cou = 3007

In [2]:
# creates pd named couid
# contains data for county id, county, county-st

In [86]:
coucsv = pd.read_csv('counties.csv')
statecsv = pd.read_csv('50_us_states_all_data.csv')

cou_st = pd.merge(statecsv,
                    coucsv,
                    on='State')

cou_st = cou_st.rename({'Unnamed: 0': 'COU-ST'}, axis='columns')

# print(any(cou_st['COU-ST'].duplicated()))

cou = cou_st['County'].tolist()
state = cou_st['Abbr'].tolist()
ind = cou_st['COU-ST'].tolist()


for i in np.arange(num_cou):
    c_st = cou[i] + ', ' + state[i]
    cou_st = cou_st.replace({'COU-ST': {ind[i]: c_st}})

In [88]:
couid = cou_st[['CountyId', 'COU-ST']]

In [89]:
# create pd named crm using csv
# clean crm

In [90]:
crm = pd.read_csv('crime_data_w_population_and_crime_rate.csv')

In [91]:
crm = crm.drop(['index', 'EDITION', 'PART'], axis=1)
cou_crm = crm[crm['county_name'].str.contains('County')]

In [92]:
cou_head = cou_crm.columns.tolist()

da_cou = cou_crm.mean(axis=0).tolist()
test = cou_crm.max(axis=0).tolist()
da_cou[0] = 'Doña Ana County, NM'

da_cou.insert(2, 10)

cou_crm = cou_crm.append(pd.DataFrame([da_cou], columns=cou_head), ignore_index=True, sort=False)
# shannon county, sd --> oglala lakota county, sd (name changed in 2015)
cou_crm = cou_crm.replace({'county_name': {'Shannon County, SD': 'Oglala Lakota County, SD'}})

In [94]:
cou_crm = pd.merge(couid,
                 cou_crm,
                 left_on='COU-ST',
                 right_on='county_name')

In [95]:
cou_names = {}

for cou in crm.iloc[:,0]:
    cou_names[cou] = cou[:-4]

cou_crm = cou_crm.replace({'county_name': cou_names})

In [106]:
cou_crm

,CountyId,COU-ST,county_name,crime_rate_per_100000,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,COVIND,...,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY
0,1001,"Autauga County, AL",Autauga County,251.601926,1,57161.0,54604.0,3.0,3.0,100.0000,...,15.0,34.0,87.0,447.0,1233.0,85.0,108.0,55246.0,1.0,1.0
1,1003,"Baldwin County, AL",Baldwin County,228.086325,2,187467.0,187467.0,14.0,14.0,100.0000,...,30.0,76.0,332.0,967.0,3829.0,192.0,31.0,195540.0,1.0,3.0
2,1005,"Barbour County, AL",Barbour County,177.278771,3,27228.0,25566.0,6.0,4.0,97.0113,...,4.0,8.0,36.0,90.0,362.0,21.0,0.0,27076.0,1.0,5.0
3,1007,"Bibb County, AL",Bibb County,217.661692,4,22907.0,22857.0,5.0,6.0,94.4853,...,4.0,8.0,36.0,122.0,251.0,27.0,0.0,22512.0,1.0,7.0
4,1009,"Blount County, AL",Blount County,210.810064,5,57909.0,57909.0,6.0,6.0,98.6338,...,11.0,9.0,101.0,397.0,865.0,86.0,9.0,57872.0,1.0,9.0
5,1011,"Bullock County, AL",Bullock County,526.365260,6,10584.0,10584.0,2.0,2.0,57.5822,...,2.0,12.0,43.0,98.0,55.0,3.0,23.0,10639.0,1.0,11.0
6,1013,"Butler County, AL",Butler County,537.873180,7,20741.0,20733.0,4.0,4.0,97.6483,...,5.0,11.0,87.0,182.0,408.0,30.0,3.0,20265.0,1.0,13.0
7,1015,"Calhoun County, AL",Calhoun County,580.797697,8,122213.0,118270.0,8.0,10.0,99.5075,...,37.0,131.0,504.0,1710.0,3482.0,147.0,34.0,116736.0,1.0,15.0
8,1017,"Chambers County, AL",Chambers County,562.027984,9,34075.0,34075.0,4.0,4.0,100.0000,...,7.0,24.0,160.0,374.0,875.0,96.0,5.0,34162.0,1.0,17.0
9,1019,"Cherokee County, AL",Cherokee County,415.982903,10,26141.0,26162.0,4.0,5.0,100.0000,...,4.0,2.0,102.0,329.0,610.0,75.0,12.0,26203.0,1.0,19.0


In [108]:
result = pd.merge(coucsv,
                  cou_crm,
                  on='CountyId').drop(['Unnamed: 0', 'COU-ST', 'county_name', 'population', 'IDNO'], axis=1)
result = result.rename({'crime_rate_per_100000': 'CrmRatePer100k'}, axis='columns')

print(result.columns.tolist())
print('')
result[['INDEX']]

['CountyId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen', 'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction', 'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork', 'SelfEmployed', 'FamilyWork', 'Unemployment', 'CrmRatePer100k', 'CPOPARST', 'CPOPCRIM', 'AG_ARRST', 'AG_OFF', 'COVIND', 'INDEX', 'MODINDX', 'MURDER', 'RAPE', 'ROBBERY', 'AGASSLT', 'BURGLRY', 'LARCENY', 'MVTHEFT', 'ARSON', 'FIPS_ST', 'FIPS_CTY']



,INDEX
0,139.0
1,446.0
2,48.0
3,49.0
4,122.0
5,56.0
6,109.0
7,678.0
8,192.0
9,109.0


In [ ]:
# TESTS BELOW

In [ ]:
# test to find the missing county
# missing county = Doña Ana County

In [ ]:
c_test = crm[crm['county_name'].str.contains('County')]['county_name']
c1_test = c_test.sort_values(ascending=True).reset_index()['county_name'].tolist()
c1_test

test01 = pd.read_csv('counties.csv')
test02 = test01['County'].sort_values(ascending=True).reset_index()['County'].tolist()

for t in np.arange(len(c1_test)):
    if c1_test[t] != test02[t]:
        print(test02[t])
        print(c1_test[t])
        break

pd_test.to_csv('ree')

test01[test01['County'].str.contains('Ana')]